#### Downscaled Climate Projections Archive processing
###### Workflow
* Submit data request
* Download data to data folder
* Process NetCDF files into dataframes
* Compute annual means (2000, 2005, and 2010) from monthly data
* Compute means for each state

In [5]:
#Import libraries
import sys, os, glob, time, datetime
import numpy as np
import pandas as pd
import netCDF4
import matplotlib
%matplotlib inline

In [6]:
#Get nc filenames
dataFolder = r'C:\Workspace\Gits\WaterAccounting\Data\hydro5\*.nc'
dataFiles = glob.glob(dataFolder)
dataFile = dataFiles[0]
print dataFile

C:\Workspace\Gits\WaterAccounting\Data\hydro5\Extraction_et.nc


In [7]:
#Convert NetCDF to pandas dataframe for each year
nc = netCDF4.Dataset(dataFile,mode='r')
lats = nc.variables['latitude'][:]
lons = nc.variables['longitude'][:]
times = nc.variables['time']
et = nc.variables['et']

In [ ]:
#Format of the et variable
print et.shape
#21 different projections
#132 months (Jan 2000 thru Dec 2010)
#216 N-S samples (latitudes)
#460 E-W samples (longitudes)

In [20]:
#Make yearly means of the ccsm4.1.rcp26 projection (idx = 2)

#Set the index of the climate model to use -- see projections5.txt
rcpIndex = 2 #ccsm4.1.rcp26 

t0 = 0
et2000 = et[rcpIndex,t0:t0+12,:,:].mean(axis=0)

t5 = 5 * 12
et2005 = et[rcpIndex,t5:t5+12,:,:].mean(axis=0)

t10 = 10 * 12
et2010 = et[rcpIndex,t10:t10+12,:,:].mean(axis=0)

print et2000.shape, et2005.shape, et2010.shape
#This leaves us with 3 ndarrays: one for each year with et values at each lat/long

(216, 460) (216, 460) (216, 460)


In [38]:
#Export as table of x,y, et
outFile = open("foo.csv",'wt')
outFile.write("Longitude,Latitude,ET2000,ET2005,ET2010\n")
for x in xrange(lons.size):
    for y in xrange(lats.size):
        etVal00 = et2000[y,x]
        etVal05 = et2005[y,x]
        etVal10 = et2010[y,x]
        outStr = "{},{},{},{},{}\n".format(lons[x],lats[y],etVal00,etVal05,etVal10)
        if type(et2005[y,x]) is not np.ma.core.MaskedConstant: 
            outFile.write(outStr)
outFile.close()
        

In [34]:
type(et2005[x,y]) is not np.ma.core.MaskedConstant

False

In [ ]:
np.savetxt("foo.csv",np.asarray(et2000)[:,:],delimiter=',')
#df = pd.DataFrame(et2000.tolist())
#np.savetxt("foo.csv",np.asarray(et2000))

In [ ]:
print df.shape
print et2000.shape

In [ ]:
print times.units
print times[-1]
start = datetime.date(1950,1,1)
delta = datetime.timedelta(int(times[72]))
offset = start + delta
print offset